<a href="https://www.kaggle.com/code/mikedelong/linear-regression-r2-0-2334?scriptVersionId=161329864" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
from warnings import filterwarnings
# plotly and pandas aren't playing nice right now when we use multiple colors
filterwarnings(action='ignore', category=FutureWarning) 

THAMES = '/kaggle/input/thames-water-daily-sewage-spills/thames_water_aggregated_daily_sewage_spills.csv'

df = pd.read_csv(filepath_or_buffer=THAMES, parse_dates=['start_day'], date_format='%d/%m/%Y')
df.head()

,start_day,chenies_rain,crossness_rain,edenvale_rain,haveringbower_rain,hollandpark_rain,leatherhead_rain,northmymms_rain,num_active_events,num_unique_locs,num_start_events,median_spill_duration,mean_spill_duration,sum_spill_duration
0,2022-12-12,0.0,4.4,1.14,1.38,1.4,0.0,1.6,2,2,2,44.5,44.5,89
1,2022-12-13,0.2,2.0,0.28,0.00,0.6,1.2,0.2,1,1,1,540.0,540.0,540
2,2022-12-14,0.0,0.0,4.86,0.00,0.0,0.0,0.2,0,0,0,0.0,0.0,0
3,2022-12-15,0.2,0.0,1.18,0.69,0.2,0.0,1.6,0,0,0,0.0,0.0,0
4,2022-12-16,1.2,0.2,0.02,0.02,0.0,0.0,1.0,0,0,0,0.0,0.0,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   start_day              366 non-null    datetime64[ns]
 1   chenies_rain           366 non-null    float64       
 2   crossness_rain         366 non-null    float64       
 3   edenvale_rain          366 non-null    float64       
 4   haveringbower_rain     366 non-null    float64       
 5   hollandpark_rain       366 non-null    float64       
 6   leatherhead_rain       366 non-null    float64       
 7   northmymms_rain        366 non-null    float64       
 8   num_active_events      366 non-null    int64         
 9   num_unique_locs        366 non-null    int64         
 10  num_start_events       366 non-null    int64         
 11  median_spill_duration  366 non-null    float64       
 12  mean_spill_duration    366 non-null    float64       
 13  sum_s

In [3]:
from plotly.express import line
line(data_frame=df, x='start_day', y=[column for column in df.columns if column.endswith('rain')])

This is pretty but we should probably render it as a scatter plot because rainfall is fundamentally discontinuous.

In [4]:
from plotly.express import scatter
rain_columns = [column for column in df.columns if column.endswith('rain')]
scatter(data_frame=df, x='start_day', y=rain_columns, trendline='ols', trendline_scope='overall')

Our trendline is hard to interpret in terms of the available variables, but it appears to be saying we get 1.9-2.6 units of rain all the time on average.

In [5]:
duration_columns = [column for column in df.columns if column.endswith('duration')]
scatter(data_frame=df, x='start_day', y=duration_columns, log_y=True)

We have a choice of target variables; unfortunately the spill duration (the output variable) and the rain (the input variables) are probably provided in different units and we don't know what they are. 

In [6]:
from plotly.express import imshow
imshow(img=df[rain_columns + duration_columns].corr())

This is a little encouraging: one of our duration columns seems to have a higher correlation with the rain columns than the others. Let's build a simple model and see.

But first let's check the autocorrelation of our data; the fact that we have daily data is probably an issue.

In [7]:
from plotly.express import histogram
histogram(x=rain_columns + duration_columns, y=[df[column].autocorr(lag=1) for column in rain_columns + duration_columns])

Our one-day autocorrelations are quite low; and furthermore our inputs and outputs have rather different values.

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
TARGET = 'sum_spill_duration'

X_train, X_test, y_train, y_test = train_test_split(df[rain_columns], df[TARGET].values, test_size=0.2, random_state=2024)
model = LinearRegression().fit(X=X_train, y=y_train)

print('R2: {:6.4f}'.format(r2_score(y_true=y_test, y_pred = model.predict(X=X_test))))

R2: 0.2334


Our r2 is poor but not surprising: we are missing important data that would tell us a lot about the physics of the Thames.

Let's put our test and predicted data back into a time series and see if we can tell anything about our result.

In [9]:
result_df = pd.DataFrame(data={'date': pd.to_datetime(df.iloc[X_test.index]['start_day'].values).tolist(), 'true' : y_test.tolist(), 
                               'predicted': model.predict(X=X_test).tolist()}).sort_values(by='date').reset_index(drop=True)
line(data_frame=result_df, x='date', y=['true', 'predicted']).show()

Not surprisingly we are overshooting when the values are low and undershooting when values are large. If we had more granular data (hourly instead of daily) presumably this would be less of a problem.